<a href="https://colab.research.google.com/github/sourajitaghosh/TirePredictionRecommendation/blob/main/TireRecommendationAI_SG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import random

# Define a list of customer names
customer_names = ["Alice Johnson", "Bob Smith", "Charlie Brown", "Diana Garcia", "Edward Miller", "Fiona Lee", "George Williams", "Helen Jones", "Irene Lopez", "James Brown"]

# Define a list of vehicle types
vehicles = ["Sedan", "SUV", "Truck", "Coupe", "Hatchback", "Minivan", "Motorcycle"]

# Define a list of tire brands
tire_brands = ["Michelin", "Bridgestone", "Goodyear", "Continental", "Pirelli", "Dunlop", "Hankook"]

# Define a list of weather details
weather_details = ["Sunny", "Rainy", "Snowy", "Icy", "Cloudy", "Foggy"]

# Define a list of additional parameters for tire selection and tread matching
additional_parameters = [
    "Mileage rating",
    "Load capacity",
    "Speed rating",
    "Treadwear warranty",
    "Noise level",
    "Fuel efficiency",
    "Comfort",
    "Handling",
    "Grip",
    "Price",
]

# Generate 50 rows of data
data = []
for _ in range(50):
    customer_name = random.choice(customer_names)
    vehicle = random.choice(vehicles)
    tire_brand = random.choice(tire_brands)
    weather_detail = random.choice(weather_details)
    loyalty_points = random.randint(0, 1000)

    # Generate random values for additional parameters
    additional_data = [random.randint(1, 10) for _ in range(len(additional_parameters))]

    row = [customer_name, vehicle, tire_brand, weather_detail, loyalty_points] + additional_data
    data.append(row)

# Write the data to a CSV file
with open("tire_management_data.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Name of customer", "Vehicle", "Tire brand", "Weather details", "Loyalty points"] + additional_parameters)
    writer.writerows(data)

print("CSV file generated successfully!")


CSV file generated successfully!


In [3]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/tire_management_data.csv /content/drive/MyDrive/
from google.colab import files
files.download("/content/drive/MyDrive/tire_management_data.csv")


Mounted at /content/drive


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# Load the tire data
#importing dataset
url = 'https://raw.githubusercontent.com/sourajitaghosh/TirePredictionRecommendation/main/tire_management_data.csv?token=GHSAT0AAAAAACNFAWSZV452HN6W3M27IAKGZNNMBOA'
data = pd.read_csv(url)
# Dataset is now stored in a Pandas Dataframe

# Select relevant columns for content-based filtering
selected_columns = [
    'Name of customer', 'Vehicle', 'Tire brand', 'Weather details',
    'Loyalty points', 'Mileage rating', 'Load capacity', 'Speed rating',
    'Treadwear warranty', 'Noise level', 'Fuel efficiency',
    'Comfort', 'Handling', 'Grip', 'Price'
]
data = data[selected_columns]

# Clean the data (optional)
data = data.fillna('')

#Content based filtering

# Combine selected columns into a single 'description' column
data['description'] = data.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Apply TF-IDF vectorization on tire descriptions
tfidf_matrix = tfidf_vectorizer.fit_transform(data['description'])

# Calculate the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Function to get top N similar tires based on cosine similarity
def get_similar_tires(tire_brand, N=5):
    # Get the index of the tire brand
    indices = pd.Series(data.index, index=data['Tire brand']).drop_duplicates()
    idx = indices[tire_brand]

    # Calculate the similarity scores for all tires
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the tires based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1][0], reverse=True)

    # Get the top N similar tires
    top_tires_indices = [i[0] for i in sim_scores[1:N+1]]
    top_tires = data['Tire brand'].iloc[top_tires_indices]

    return top_tires

# Example usage: Get top 5 similar tires to a specific brand
similar_tires = get_similar_tires("Hankook", N=5)
print(similar_tires)


0          Dunlop
7         Pirelli
4        Goodyear
11    Bridgestone
6        Michelin
Name: Tire brand, dtype: object


In [14]:
# Collaborative filtering

# Create a user-item matrix
user_item_matrix = data.pivot_table(index='Name of customer', columns='Tire brand', values='Comfort')

# Calculate the similarity between users using cosine similarity
user_similarity = cosine_similarity(user_item_matrix.fillna(0))

# Function to generate recommendations for a user based on collaborative filtering
def generate_recommendations(user_name, num_recommendations):
    # Find the top N similar users to the given user
    user_id = user_item_matrix.index.get_loc(user_name)
    similar_users = np.argsort(user_similarity[user_id])[::-1][1:num_recommendations+1]

    # Get the tires used and rated by the similar users
    similar_users_ratings = user_item_matrix.iloc[similar_users].dropna(axis=1, how='all')

    # Calculate the average rating of each tire
    tire_avg_ratings = similar_users_ratings.mean()

    # Filter out the tires already used and rated by the user
    used_tires = user_item_matrix.loc[user_name].dropna().index
    tire_avg_ratings = tire_avg_ratings[~tire_avg_ratings.index.isin(used_tires)]

    # Sort the tires based on average rating in descending order
    recommendations = tire_avg_ratings.sort_values(ascending=False)[:num_recommendations]

    return recommendationshttps://drive.google.com/drive/search?q=owner%3Ame%20(type%3Aapplication%2Fvnd.google.colaboratory%20%7C%7C%20type%3Aapplication%2Fvnd.google.colab)&authuser=0

# Generate recommendations for a specific user
user_name = 'Edward Miller'
num_recommendations = 5
recommendations = generate_recommendations(user_name, num_recommendations)

print(f"Top {num_recommendations} recommendations for User {user_name}:")
print(recommendations)


Top 5 recommendations for User Edward Miller:
Tire brand
Bridgestone    6.000000
Continental    6.000000
Pirelli        5.375000
Hankook        5.166667
Goodyear       3.500000
dtype: float64
